In [1]:
!git clone https://github.com/stipot/image_processing

Cloning into 'image_processing'...
remote: Enumerating objects: 2321, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 2321 (delta 23), reused 156 (delta 16), pack-reused 2152 (from 1)
Receiving objects: 100% (2321/2321), 125.24 MiB | 10.37 MiB/s, done.
Resolving deltas: 100% (24/24), done.
Updating files: 100% (2888/2888), done.


# Модуль 2. Классификация изображений


## Сопоставление изображений


### Алгоритмы сопоставления изображений

Для реализации алгоритма классификации изображений, я предложу вам пример кода, использующий библиотеку tensorflow и keras. В этом примере мы будем использовать предварительно обученную модель MobileNetV2 для классификации изображений.

Прежде всего, убедитесь, что у вас установлены tensorflow и keras


In [3]:
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import (
    preprocess_input,
    decode_predictions,
)
from tensorflow.keras.preprocessing import image

# Загрузка предварительно обученной модели MobileNetV2
model = MobileNetV2(weights="imagenet")


def classify_image(img_path):
    # Загрузка изображения, его предварительная обработка и расширение размерностей
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    preprocessed_image = preprocess_input(img_array_expanded_dims)

    # Прогнозирование и декодирование предсказаний
    predictions = model.predict(preprocessed_image)
    results = decode_predictions(predictions, top=3)[0]

    # Вывод результатов
    for i, (imagenet_id, label, score) in enumerate(results):
        print(f"{i + 1}: {label} ({score*100:.2f}%)")


# Пример использования
img_path = "texture_01.jpg"
classify_image(img_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1: fur_coat (10.58%)
2: bittern (2.80%)
3: tiger (1.93%)


### Построение мозаики из изображений

Для построения мозаики изображений на Python, можно использовать библиотеку PIL (Python Imaging Library), которая теперь доступна как Pillow. Ниже приведен пример кода, который создает мозаику из нескольких изображений. Допустим, у нас есть четыре изображения, и мы хотим их объединить в одно большое изображение-мозаику в формате 2x2.

Прежде всего, убедитесь, что у вас установлена библиотека Pillow


In [5]:
from PIL import Image


def crop_to_square(img, size):
    """
    Обрезка изображения до квадратной формы по центру.
    """
    # Определение текущих размеров
    width, height = img.size

    # Определение размера для обрезки
    new_size = min(width, height, size)

    # Вычисление координат для обрезки
    left = (width - new_size) / 2
    top = (height - new_size) / 2
    right = (width + new_size) / 2
    bottom = (height + new_size) / 2

    # Обрезка и возврат изображения
    img = img.crop((left, top, right, bottom))
    return img.resize((size, size), Image.Resampling.LANCZOS)


# Пути к изображениям, которые будут включены в мозаику
image_paths = ["image1.jpg", "image2.jpg", "image3.jpg", "image4.jpg"]

# Открытие изображений и их сохранение в список
images = [Image.open(f"{x}") for x in image_paths]

# Определение минимального размера среди всех изображений
min_size = min(min(img.size) for img in images)

# Обрезка изображений до квадратов и изменение их размера
squared_images = [crop_to_square(img, min_size) for img in images]

# Создание нового пустого изображения для мозаики
mosaic_size = (min_size * 2, min_size * 2)
mosaic_image = Image.new("RGB", mosaic_size, (255, 255, 255))

# Размещение изображений в мозаике
positions = [(0, 0), (min_size, 0), (0, min_size), (min_size, min_size)]
for img, pos in zip(squared_images, positions):
    mosaic_image.paste(img, pos)

# Сохранение и показ мозаики
mosaic_image.save("./result/mosaic.jpg")
mosaic_image.show()

FileNotFoundError: [Errno 2] No such file or directory: '/content/result/mosaic.jpg'

### Построение панорамных изображений


In [6]:
import cv2
import numpy as np


def create_panorama(image1_path, image2_path, output_path="panorama.jpg"):
    # Загружаем изображения
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)

    # Инициализируем ORB детектор
    orb = cv2.ORB_create()

    # Находим ключевые точки и дескрипторы с помощью ORB
    keypoints1, descriptors1 = orb.detectAndCompute(image1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(image2, None)

    # Создаем объект BFMatcher и совершаем сопоставление дескрипторов
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)

    # Сортируем сопоставления по расстоянию (лучшие сопоставления первые)
    matches = sorted(matches, key=lambda x: x.distance)

    # Извлекаем координаты соответствующих ключевых точек для сопоставленных дескрипторов
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    # Находим матрицу гомографии
    H, _ = cv2.findHomography(points1, points2, cv2.RANSAC)

    # Применяем преобразование гомографии к изображению 1
    height, width, channels = image2.shape
    panorama = cv2.warpPerspective(image1, H, (width * 2, height))

    # Копируем изображение 2 в панорамное изображение
    panorama[0 : image2.shape[0], 0 : image2.shape[1]] = image2

    # Сохраняем панорамное изображение
    cv2.imwrite(output_path, panorama)


# Пример использования:
create_panorama("panorama02.jpg", "panorama01.jpg", "output_panorama.jpg")

## Распознавание лиц


In [7]:
!pip install face_recognition
import face_recognition
import matplotlib.pyplot as plt

face = ["3faces.jpg", "manyfaces.jpg"]
# Загрузка изображения
image_path = f"./data/faces/{face[0]}"
image = face_recognition.load_image_file(image_path)

# Нахождение лиц на изображении
face_locations = face_recognition.face_locations(image)

# Вывод результатов
print(f"Найдено лиц: {len(face_locations)}")
plt.imshow(image)
for face_location in face_locations:
    top, right, bottom, left = face_location
    plt.plot([left, right, right, left, left], [top, top, bottom, bottom, top], "r-")
plt.axis("off")
plt.show()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566164 sha256=7e00a88293309c29137b6dbf906950966fcc7b4b73e5f8845d35e057714093ee
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


RuntimeError: Error while calling cudaGetDevice(&the_device_id) in file /tmp/pip-install-don9buxi/dlib_1f308c2e29ca4ef1ad3bf1b73c1cf6b9/dlib/cuda/gpu_data.cpp:204. code: 35, reason: CUDA driver version is insufficient for CUDA runtime version

In [8]:
from PIL import Image
import os


def resize_image(input_image_path, output_image_path, target_size):
    with Image.open(input_image_path) as image:
        original_width, original_height = image.size
        max_dimension = max(original_width, original_height)
        scale = target_size / max_dimension
        new_width = int(original_width * scale)
        new_height = int(original_height * scale)

        resized_image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)
        resized_image.save(output_image_path)


def process_images_folder(input_folder, output_folder, target_size):
    # Получение абсолютных путей папок
    script_dir = os.getcwd()
    abs_input_folder = os.path.join(script_dir, input_folder)
    abs_output_folder = os.path.join(script_dir, output_folder)

    if not os.path.exists(abs_output_folder):
        os.makedirs(abs_output_folder)

    for filename in os.listdir(abs_input_folder):
        input_image_path = os.path.join(abs_input_folder, filename)
        if os.path.isfile(input_image_path):
            output_image_path = os.path.join(abs_output_folder, filename)
            try:
                resize_image(input_image_path, output_image_path, target_size)
                print(f"Image {filename} resized and saved to {output_folder}")
            except Exception as e:
                print(f"Error processing file {filename}: {e}")


# Пути к папкам относительно скрипта
input_folder = "./data/plibrary/src"
output_folder = "./data/plibrary/resampled"
target_size = 800  # Целевой размер для большей стороны изображения

# Обработка всех изображений в папке
process_images_folder(input_folder, output_folder, target_size)

FileNotFoundError: [Errno 2] No such file or directory: '/content/./data/plibrary/src'

In [ ]:
import face_recognition
import os
from shutil import copy2


def filter_images_with_faces(source_folder, destination_folder):
    # Создание папки назначения, если она не существует
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Перебор всех файлов в исходной папке
    for filename in os.listdir(source_folder):
        file_path = os.path.join(source_folder, filename)

        # Убедитесь, что это файл и он имеет расширение изображения
        if os.path.isfile(file_path) and filename.lower().endswith((".png", ".jpg", ".jpeg")):
            try:
                # Загрузка изображения и поиск лиц
                image = face_recognition.load_image_file(file_path)
                face_locations = face_recognition.face_locations(image)

                # Если лица обнаружены, копировать файл в папку назначения
                if len(face_locations) > 0:
                    destination_path = os.path.join(destination_folder, filename)
                    copy2(file_path, destination_path)
                    print(f"Image {filename} has faces and was copied.")
            except Exception as e:
                print(f"Error processing file {filename}: {e}")


# Задайте пути к папкам
script_dir = os.getcwd()
source_folder = os.path.join(script_dir, "./data/plibrary/resampled")
destination_folder = os.path.join(script_dir, "./data/plibrary/justfaces")

# Фильтрация изображений
filter_images_with_faces(source_folder, destination_folder)

In [ ]:
import face_recognition
import os
import shutil


def create_directory_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)


def find_face_encodings(image_path):
    image = face_recognition.load_image_file(image_path)
    return face_recognition.face_encodings(image)


def are_faces_same(face_encoding, known_faces):
    # Если нет известных лиц, не сравниваем
    if len(known_faces) == 0:
        return -1
    distances = face_recognition.face_distance(known_faces, face_encoding)
    best_match_index = distances.argmin()
    if distances[best_match_index] < 0.6:
        return best_match_index
    else:
        return -1


source_folder = os.path.join(script_dir, "./data/plibrary/justfaces")
destination_folder = os.path.join(script_dir, "./data/plibrary/grouped")
create_directory_if_not_exists(destination_folder)

known_faces = []
face_folders = []

# Перебор всех файлов в исходной папке
for filename in os.listdir(source_folder):
    if filename.lower().endswith((".png", ".jpg", ".jpeg")):
        image_path = os.path.join(source_folder, filename)
        image_encodings = find_face_encodings(image_path)

        for encoding in image_encodings:
            match_index = are_faces_same(encoding, known_faces)

            if match_index != -1:
                # Лицо совпадает с известным, копируем в соответствующую папку
                shutil.copy2(image_path, face_folders[match_index])
            else:
                # Новое лицо, создаем для него папку
                new_folder_path = os.path.join(destination_folder, f"person_{len(known_faces)}")
                create_directory_if_not_exists(new_folder_path)
                shutil.copy2(image_path, new_folder_path)

                # Добавляем лицо и папку в известные
                known_faces.append(encoding)
                face_folders.append(new_folder_path)

print(f"Фотографии были сгруппированы по {len(known_faces)} лицам.")

## Коллекции для обучения


In [ ]:
from sklearn.datasets import fetch_lfw_people
import matplotlib.pyplot as plt

# Загрузка набора данных LFW
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

# Получение ссылок на изображения и их метки
images = lfw_people.images
target_names = lfw_people.target_names
targets = lfw_people.target

# Отображение первых 3 изображений
fig, ax = plt.subplots(1, 3, figsize=(10, 5))
for i in range(3):
    ax[i].imshow(images[i], cmap="gray")
    ax[i].set_title(target_names[targets[i]])
    ax[i].axis("off")
plt.show()

## Квантование многомерных признаков

КМП - это процесс преобразования непрерывных или многомерных признаков в дискретные значения, часто используемый для уменьшения количества различных признаков и упрощения алгоритмов машинного обучения, в том числе в контексте классификации изображений и поиска похожих изображений.

В контексте классификации изображений и поиска похожих изображений, одним из распространенных подходов является использование векторов признаков, извлеченных из изображений с помощью предварительно обученных моделей глубокого обучения (например, сетей на основе архитектуры CNN). Квантование этих векторов признаков позволяет сократить объем хранимых данных и ускорить процесс сравнения изображений.

В качестве примера, давайте рассмотрим использование предварительно обученной модели CNN для извлечения признаков из изображений и их последующее квантование с помощью алгоритма K-средних (K-means). Затем мы можем использовать эти квантованные признаки для поиска похожих изображений.


In [ ]:
import os
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from sklearn.cluster import KMeans

# Загрузка предварительно обученной модели ResNet50 без верхнего слоя
base_model = ResNet50(weights="imagenet", include_top=False, pooling="avg")
model = Model(inputs=base_model.input, outputs=base_model.output)


def extract_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    features = model.predict(preprocessed_img)
    return features.flatten()


def get_features_from_folder(folder_path):
    features_list = []
    images_paths = []
    for img_name in os.listdir(folder_path):
        if img_name.lower().endswith(("png", "jpg", "jpeg")):
            img_path = os.path.join(folder_path, img_name)
            features = extract_features(img_path)
            features_list.append(features)
            images_paths.append(img_path)
    return features_list, images_paths


# Извлекаем признаки из всех изображений в папке
folder_path = os.path.join(script_dir, "./data/plibrary/justfaces")
features_list, images_paths = get_features_from_folder(folder_path)

# Применяем K-средние для квантования признаков
num_clusters = 3  # Примерное количество ожидаемых групп
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(features_list)

# Находим индекс центроида для каждого изображения
labels = kmeans.labels_

# Группировка изображений по кластерам
clustered_images = {}
for i, label in enumerate(labels):
    clustered_images.setdefault(label, []).append(images_paths[i])

# Вывод результатов группировки
""" for cluster_id, images in clustered_images.items():
    print(f"Cluster {cluster_id}:")
    for img_path in images:
        print(f" - {img_path}") """
# Выведем на экран
import matplotlib.pyplot as plt
from PIL import Image


def create_thumbnail(image_path, thumbnail_size=(100, 100)):
    with Image.open(image_path) as img:
        img.thumbnail(thumbnail_size)
        return img


def display_clustered_images(clustered_images, images_per_row=5):
    for cluster_id, images in clustered_images.items():
        print(f"Cluster {cluster_id} contains {len(images)} images.")
        num_rows = int(np.ceil(len(images) / images_per_row))
        num_cols = min(images_per_row, len(images))

        if len(images) == 1:
            fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
            axs = np.array([[axs]])  # Обеспечиваем 2D массив для единообразия
        else:
            fig, axs = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(15, 3 * num_rows))
            axs = np.atleast_2d(axs)  # Убеждаемся, что axs всегда 2D массив

        # Преобразуем axs в 1D массив для упрощения итерации
        axs_flat = axs.ravel()

        for ax, img_path in zip(axs_flat, images):
            thumbnail = create_thumbnail(img_path)
            ax.imshow(thumbnail)
            ax.axis("off")

        # Скрываем пустые ячейки
        for ax in axs_flat[len(images) :]:
            ax.axis("off")

        plt.tight_layout()
        plt.show()


# Отображаем изображения по кластерам
display_clustered_images(clustered_images)

### Извлечение признаков с помощью Пространственной Пирамиды

Для классификации изображений с использованием пространственной пирамиды, мы адаптируем предыдущий алгоритм, добавив этап создания пространственной пирамиды изображений перед кластеризацией с помощью K-средних. Это позволит нам учитывать локальные признаки на разных масштабах, что может улучшить точность классификации.


In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np


def extract_pyramid_features(img_path, model, levels=[1, 2, 4]):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Исходные размеры изображения
    base_height, base_width = img_array.shape[1:3]

    features = []
    for level in levels:
        for y in range(level):
            for x in range(level):
                # Вычисление координат области для текущего уровня
                width = base_width // level
                height = base_height // level
                x_start = x * width
                y_start = y * height

                # Вырезаем часть изображения и извлекаем признаки
                img_crop = img_array[:, y_start : y_start + height, x_start : x_start + width, :]
                crop_features = model.predict(img_crop)
                features.append(crop_features.flatten())

    # Объединение признаков со всех уровней
    final_features = np.concatenate(features)
    return final_features


# Загрузка модели
model = ResNet50(weights="imagenet", include_top=False, pooling="avg")

# Пример извлечения признаков
img_path = "./data/plibrary/justfaces/img_09.jpg"
features = extract_pyramid_features(img_path, model)

print(f"Извлеченные признаки: {features.shape}")

In [ ]:
def extract_pyramid_features(img_path, model, levels=[1, 2, 4]):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    features = []
    for level in levels:
        scale = 224 // level
        for y in range(level):
            for x in range(level):
                # Изменение размера и обрезка для создания пирамиды
                resized_img = image.smart_resize(img_array, (scale * level, scale * level))
                cropped_img = resized_img[:, y * scale : (y + 1) * scale, x * scale : (x + 1) * scale, :]
                # Извлечение признаков
                feature = model.predict(cropped_img).flatten()
                features.append(feature)

    # Объединение признаков со всех уровней
    final_features = np.concatenate(features)
    return final_features


# Загрузка предварительно обученной модели
model = ResNet50(weights="imagenet", include_top=False, pooling="avg")

# Извлечение признаков из всех изображений в папке
features_list, images_paths = get_features_from_folder(folder_path)

# Квантование признаков
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(features_list)

# Находим индекс центроида для каждого изображения
labels = kmeans.labels_

# Группировка изображений по кластерам
clustered_images = {}
for i, label in enumerate(labels):
    clustered_images.setdefault(label, []).append(images_paths[i])

# Отображаем изображения по кластерам
display_clustered_images(clustered_images)

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from sklearn.cluster import KMeans

# Загрузка предварительно обученной модели ResNet50 без верхнего слоя
base_model = ResNet50(weights="imagenet", include_top=False, pooling="avg")
model = Model(inputs=base_model.input, outputs=base_model.output)


def extract_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    features = model.predict(preprocessed_img)
    return features.flatten()


def extract_pyramid_features(img_path, model, levels=[1, 2, 4]):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)

    features = []
    for level in levels:
        level_features = []
        step_size = 224 // level
        for y in range(level):
            for x in range(level):
                crop = preprocessed_img[:, y * step_size : (y + 1) * step_size, x * step_size : (x + 1) * step_size, :]
                crop_features = model.predict(crop).flatten()
                level_features.append(crop_features)
        features.extend(level_features)
    return np.concatenate(features)


def get_features_from_folder(folder_path):
    features_list = []
    images_paths = []
    for img_name in os.listdir(folder_path):
        if img_name.lower().endswith(("png", "jpg", "jpeg")):
            img_path = os.path.join(folder_path, img_name)
            features = extract_pyramid_features(img_path, model)
            features_list.append(features)
            images_paths.append(img_path)
    return features_list, images_paths


# Извлекаем признаки из всех изображений в папке
script_dir = os.getcwd()
folder_path = os.path.join(script_dir, "./data/plibrary/justfaces")
features_list, images_paths = get_features_from_folder(folder_path)

# Применяем K-средние для квантования признаков
num_clusters = 4  # Примерное количество ожидаемых групп
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(features_list)

# Находим индекс центроида для каждого изображения
labels = kmeans.labels_

# Группировка изображений по кластерам
clustered_images = {}
for i, label in enumerate(labels):
    clustered_images.setdefault(label, []).append(images_paths[i])


def display_clustered_images(clustered_images, images_per_row=5):
    for cluster_id, images in clustered_images.items():
        print(f"Cluster {cluster_id} contains {len(images)} images.")
        num_rows = int(np.ceil(len(images) / images_per_row))
        num_cols = min(images_per_row, len(images))

        if len(images) == 1:
            fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
            axs = np.array([[axs]])  # Обеспечиваем 2D массив для единообразия
        else:
            fig, axs = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(15, 3 * num_rows))
            axs = np.atleast_2d(axs)  # Убеждаемся, что axs всегда 2D массив

        # Преобразуем axs в 1D массив для упрощения итерации
        axs_flat = axs.ravel()

        for ax, img_path in zip(axs_flat, images):
            thumbnail = create_thumbnail(img_path)
            ax.imshow(thumbnail)
            ax.axis("off")

        # Скрываем пустые ячейки
        for ax in axs_flat[len(images) :]:
            ax.axis("off")

        plt.tight_layout()
        plt.show()


# Вывод результатов группировки
display_clustered_images(clustered_images)

# Работа с векторной БД


In [ ]:
from annoy import AnnoyIndex
import numpy as np

# Предполагаем, что extract_pyramid_features и model уже определены
feature_dim = 43008  # Задайте размерность вашего вектора признаков
index = AnnoyIndex(feature_dim, "euclidean")  # Используем Евклидово расстояние

features_list, images_paths = get_features_from_folder(folder_path)
for i, features in enumerate(features_list):
    index.add_item(i, features)

index.build(10)  # 10 деревьев
index.save("./data/image_features.ann")

In [ ]:
# Загрузка индекса
index = AnnoyIndex(feature_dim, "euclidean")
index.load("./data/image_features.ann")

# Извлечение признаков для целевого изображения
target_features = extract_pyramid_features("./data/plibrary/justfaces/img_06.jpg", model)

# Поиск ближайших соседей
nearest_ids = index.get_nns_by_vector(target_features, 5)  # Ищем 5 ближайших соседей

# Получение путей к ближайшим изображениям
nearest_image_paths = [images_paths[i] for i in nearest_ids]

print(nearest_image_paths)
from PIL import Image
import matplotlib.pyplot as plt


def display_images(image_paths):
    fig, axes = plt.subplots(1, len(image_paths), figsize=(20, 10))
    for ax, img_path in zip(axes, image_paths):
        img = Image.open(img_path)
        ax.imshow(img)
        ax.axis("off")
    plt.show()


display_images(nearest_image_paths)

In [9]:
!git add

fatal: not a git repository (or any of the parent directories): .git


In [10]:
!git commit -m 'Часть модуля 2'

fatal: not a git repository (or any of the parent directories): .git


In [11]:
!git push origin main

fatal: not a git repository (or any of the parent directories): .git


In [13]:
!git push origin main

fatal: not a git repository (or any of the parent directories): .git


In [14]:
!cd

In [15]:
!git


usage: git [--version] [--help] [-C <path>] [-c <name>=<value>]
           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]
           [-p | --paginate | -P | --no-pager] [--no-replace-objects] [--bare]
           [--git-dir=<path>] [--work-tree=<path>] [--namespace=<name>]
           [--super-prefix=<path>] [--config-env=<name>=<envvar>]
           <command> [<args>]

These are common Git commands used in various situations:

start a working area (see also: git help tutorial)
   clone     Clone a repository into a new directory
   init      Create an empty Git repository or reinitialize an existing one

work on the current change (see also: git help everyday)
   add       Add file contents to the index
   mv        Move or rename a file, a directory, or a symlink
   restore   Restore working tree files
   rm        Remove files from the working tree and from the index

examine the history and state (see also: git help revisions)
   bisect    Use binary search to find th

In [17]:
%cd image_processing

[Errno 2] No such file or directory: 'image_processing'
/content/image_processing


In [ ]:
!git add